# CentOS 安装

## 磁盘分区
**Standard Partition**
 - /boot 为引导分区，一般设置值为100M-500M。 
 - swap分区分内存的两倍（如果内存大于等于4GB，swap分8GB即可，因为分多了也是浪费磁盘空间）实现虚拟内存，建议大小是物理内存的1~2倍
 - /分区分100GB（大小根据情况自己设置）
 - 剩余的空间给/data分区，可以不单独分/data分区。

## 网络设置

### 网络参数
- 子网掩码：NETMASK & IP ==> 网段
- 网关：GETEWAY，网络的总出口
- DNS：域名解析服务器
    ![域名服务工作流程](./images/day1/域名服务的工作流程.png)

### 三种虚拟网络模式

![nat 模式](./images/day1/NAT虚拟网络配置.png)
***
![bridge 模式和 hostonly 模式](./images/day1/桥接和hostonly虚拟网络配置.png)

### 自动获取 IP
``` shell
dhclient
```
### 查看 IP
``` shell
ip addr
```
### root 编辑网络设置
``` shell
vi /etc/sysconfig/network-scripts/ifcfg-ens33
```
``` txt
TYPE=Ethernet
PROXY_METHOD=none
BROWSER_ONLY=no
# nat 设置静态 IP
BOOTPROTO=static
DEEROUTE=yes
IPV4_FAILURE_FATAL=no
IPV6INIT=yes
IPV6_AUTOCONF=yes
IPV6_DEFROUTE=yes
IPV6_FAILURE_FATAL=no
IPV6_ADDR_GEN_MODE=stable-privacy
NAME=ens33
# 删除UUID,防止克隆时出现两台机器的唯一标识是一样的
DEVICE=ens33
# 开机自动启动
ONBOOT=yes
# ip
IPADDR=192.168.100.5
# 网关
GATEWAY=192.168.100.2
# 子网掩码
NETMASK=255.255.255.0
# 使用主的DNS，设置为网关即可
DNS1=192.168.100.2
# 备用的DNS
DNS2=8.8.8.8
```
### 重启网络服务
``` shell
systemctl restart network.service
```

## 主机名

``` shell
hostnamectl set-hostname basic
```

## 关闭防火墙、SELinux

### centos6 关闭防火墙
``` shell 
# 临时关闭
service iptables stop
# 禁止开机启动
chkconfig iptables off
```

### centos7 关闭防火墙
``` shell
# 临时关闭
systemctl stop firewalld
# 禁止开机启动
systemctl disable firewalld
```

### 如果安装的 iptables-service 关闭防火墙
``` shell
yum install -y iptables-services
# 关闭防火墙
service iptables stop
# 检查状态
service iptables status
```

### 关闭 SELinux
``` shell
# 编辑 selinux
vi /etc/selinux/config
```
    将SELINUX=enforcing 改为 SELINUX=disabled，然后重启系统

``` shell
# 查看状态
/usr/sbin/setatus
```

## centos7 yum 源设置

``` shell
yum install wget
cd /etc/yum.repos.d/
mv /etc/yum.repos.d/CentOS-Base.repo  /etc/yum.repos.d/CentOS-Base.repo.backup
wget -O /etc/yum.repos.d/CentOS-Base.repo http://mirrors.aliyun.com/repo/Centos-7.repo
yum clean all 
yum makecache
```

## 克隆

### 拍摄快照
```
关机 -> 虚拟机-> 快照 -> 拍摄快照
```

### 修改静态 IP
``` shell
vi /etc/sysconfig/network-scripts/ifcfg-ens33
# IPADDR=192.168.100.61
```

### 修改主机名
``` shell
# 查看当前系统主机名
hostname 
# 修改
vi /etc/hostname
# 另一种方法
hostnamectl set-hostname centreon
```

### hosts 配置
``` txt 
192.168.100.61 centreon
```

## 图形界面

### 安装 Gnome 包
``` shell
yum groupinstall -y "GNOME Desktop"
```

### 进入图形界面
```
安装完成后，输入 init 5 或者 startx
```

### 更新系统的运行界别
``` shell
ln -sf /lib/systemd/system/runlevel5.target /etc/systemd/system/defaul.target
```
### 自动以 root 登录 gnome 桌面
``` shell
vi /etc/gdm/custom.conf
```
在 [daemon] 下添加：
``` txt
[daemon]
AutomaticLoginEnable=True
AutomaticLogin=root
```

## RPM
### rmp 卸载包

1. 查看
``` shell
rpm -qa | grep centreon
```
``` txt
centreon-release-19.04-1.el7.centos.noarch
```

2. 卸载
``` shell
rpm -e --nodeps centreon-release-19.04-1.el7.centos.noarch
```

# 文件系统

## 目录结构

![image.png](./images/day1/linuxdir.png)

- bin  (binaries)存放二进制可执行文件
- sbin  (super user binaries)存放二进制可执行文件，只有root才能访问
- etc (etcetera)存放系统配置文件
- usr  (unix shared resources)用于存放共享的系统资源
- home 存放用户文件的根目录
- root  超级用户目录
- dev (devices)用于存放设备文件
- lib  (library)存放跟文件系统中的程序运行所需要的共享库及内核模块
- mnt  (mount)系统管理员安装临时文件系统的安装点
- boot 存放用于系统引导时使用的各种文件
- tmp  (temporary)用于存放各种临时文件
- var  (variable)用于存放运行时需要改变数据的文件

## 常用命令
``` shell
ls /     # 查看根目录下的子节点（文件夹和文件）信息
ls -al    # -a是显示隐藏文件，-l是以更详细的列表形式显示

# 切换目录
cd  /home

# 创建文件夹
mkdir aaa     # 相对路径
mkdir -p aaa/bbb/ccc # 连级
mkdir  /data    # 绝对路径

# 删除文件夹
rmdir      # 只可以删除空目录
rm -r aaa   # 可以把aaa整个文件夹及其中的所有子节点全部删除，但会询问
rm -rf aaa  # 强制删除aaa，不询问

# 修改文件夹名称/移动文件
mv aaa bbb

# 创建文件
touch  aa.1   # 创建一个空文件 aa.1
> aa.2      # 创建一个空文件 aa.2
echo "hello shell" > aa.1   # 利用重定向“>”的功能，将一条指令的输出结果写入到一个文件中，会覆盖原文件内容
echo "hello linux" >> aa.1  # 将一条指令的输出结果追加到一个文件中，不会覆盖原文件内容

# 查看文件内容
cat    somefile    # 一次性将文件内容全部输出（控制台）
more   somefile    # 可以翻页查看, 下翻一页(空格)    上翻一页（b）   退出（q）
less   somefile    # 可以翻页查看,下翻一页(空格)    上翻一页（b），上翻一行(↑)  下翻一行（↓）  可以搜索关键字（/keyword）

tail -10  install.log  # 查看文件尾部的10行
tail -f install.log    # 小f跟踪文件的唯一inode号，就算文件改名后，还是跟踪原来这个inode表示的文件
tail -F install.log    # 大F按照文件名来跟踪

head -10  install.log  # 查看文件头部的10行

# 将一个文件的内容复制到里另一个文件中
cat a.txt > b.txt     # 覆盖
cat a.txt >> b.txt    # 追加

# 拷贝
cp -r /itcast /itcast-bak   # -r 表示拷贝目录下所有文件
```

## wc
```
统计文件里面有多少单词，多少行，多少字符。

wc语法
wc [-lwm]
选项与参数：
-l  ：仅列出行；
-w  ：仅列出多少字(英文单字)；
-m  ：多少字符；
```

```shell
# 使用wc统计/etc/passwd
wc /etc/passwd
# 40   45 1719 /etc/passwd
# 40是行数，45是单词数，1719是字节数


# 统计文件
wc -w aa.1  # 统计有多少单词
wc -l aa.1  # 统计有多少行
wc -m aa.1  # 统计文件的字符数
```

## sort
```
命令对 File 参数指定的文件中的行排序，并将结果写到标准输出。
如果 File 参数指定多个文件，那么 sort 命令将这些文件连接起来，并当作一个文件进行排序。

sort语法
sort [-fbMnrtuk] [file or stdin]
选项与参数：
-f  ：忽略大小写的差异，例如 A 与 a 视为编码相同；
-b  ：忽略最前面的空格符部分；
-M  ：以月份的名字来排序，例如 JAN, DEC 等等的排序方法；
-n  ：使用『纯数字』进行排序(默认是以文字型态来排序的)；
-r  ：反向排序；
-u  ：就是 uniq ，相同的数据中，仅出现一行代表；
-t  ：分隔符，默认是用 [tab] 键来分隔；
-k  ：以那个区间 (field) 来进行排序的意思
```

```shell
# 对/etc/passwd 的账号进行排序
cat /etc/passwd | sort  # 默认以第一个数据来排序，而且默认是以字符串形式来排序

# 以第三栏来排序
cat /etc/passwd | sort -t ':' -k 3  # 默认是以字符串来排序的

# 使用数字排序
cat /etc/passwd | sort -t ':' -k 3n  # 默认是升序排序

# 倒序排序
cat /etc/passwd | sort -t ':' -k 3nr

# 对/etc/passwd,先以第六个域的第2个字符到第4个字符进行正向排序，再基于第一个域进行反向排序。
cat /etc/passwd | sort -t ':' -k 6.2,6.4 -k 1r 

# 查看/etc/passwd有多少个shell，对/etc/passwd的第七个域进行排序，然后去重
cat /etc/passwd | sort -t ':' -k 7 -u
```

## uniq
```
uniq命令可以去除排序过的文件中的重复行，因此uniq经常和sort合用。
为了使uniq起作用，所有的重复行必须是相邻的。

uniq语法
uniq [-icu]
选项与参数：
-i   ：忽略大小写字符的不同；
-c  ：进行计数
-u  ：只显示唯一的行
```

```shell
# 排序文件，默认是去重
cat testfile | sort | uniq

# 排序之后删除了重复行，同时在行首位置输出该行重复的次数
sort testfile | uniq -c

# 仅显示存在重复的行，并在行首显示该行重复的次数
sort testfile | uniq -dc

# 仅显示不重复的行
sort testfile | uniq -u
```

## cut
```
从一个文本文件或者文本流中提取文本列。

cut语法
cut -d'分隔字符' -f fields     <==用于有特定分隔字符
cut -c 字符区间             <==用于排列整齐的信息
选项与参数：
-d  ：后面接分隔字符。与 -f 一起使用；
-f  ：依据 -d 的分隔字符将一段信息分割成为数段，用 -f 取出第几段的意思；
-c  ：以字符 (characters) 的单位取出固定字符区间；
```

```shell
# 将 PATH 变量取出，找出第五个路径。
echo $PATH | cut -d ':' -f 5

# 将 PATH 变量取出，找出第三和第五个路径。
echo $PATH | cut -d ':' -f 3,5

# 将 PATH 变量取出，找出第三到最后一个路径。
echo $PATH | cut -d ':' -f 3-

# 将 PATH 变量取出，找出第一到第三个路径。
echo $PATH | cut -d ':' -f 1-3

# 将 PATH 变量取出，找出第一到第三，还有第五个路径。
echo $PATH | cut -d ':' -f 1-3,5
```

## 打包与压缩
```shell

# 打包，gzip 压缩
# 分步：
tar -cvf aa.tar aa.1
gzip aa.tar
# 一步：
tar -zcvf aa.tar.gz aa.1

# 解压gz
# 分步：
gzip -d aa.tar.gz
gunzip aa.tar.gz
# 一步：
tar -zxvf aa.tar.gz

# 打包 aaa 目录，bzip2 压缩
tar -jcvf aaa.tar.bz2 aaa

# 解压bz2
tar -jxvf aaa.tar.bz2 -C bbb     # -C 指定路径

查看压缩包内容
tar -ztvf a.tar.gz

zip/unzip
```

## vi
``` shell
# 基本用法
vi aa.2
# 1、首先会进入“一般模式”，此模式只接受各种快捷键，不能编辑文件内容
# 2、按i键，就会从一般模式进入编辑模式，此模式下，敲入的都是文件内容
# 3、编辑完成之后，按Esc键退出编辑模式，回到一般模式；
# 4、再按：，进入“底行命令模式”，输入wq命令，回车即可退出编辑
```

```
# 一般模式常用快捷键
a  在光标后一位开始插入
A  在该行的最后插入
I  在该行的最前面插入
0  文件当前行的开头
$  文件当前行的末尾
gg 直接跳到文件的首行
G  直接跳到文件的末行
1+G到第一行 
9+G到第九行 = :9
dd 删除行，如果 5d，则一次性删除光标后的5行
yy 复制当前行,复制多行，则 3y，则复制当前行附近的3行
p  粘贴
v  进入字符选择模式，选择完成后，按y复制，按p粘贴
ctrl+v  进入块选择模式，选择完成后，按y复制，按p粘贴
shift+v 进入行选择模式，选择完成后，按y复制，按p粘贴
ctrl + u (undo) vi撤销修改
ctrl + r (redo) 恢复
```

```
# 底行命令模式常用快捷键
# 查找并替换
%s/he/ab    查找文件中所有he，替换为ab (s -> substitute:替换)
/ab       查找文件中出现的you，并定位到第一个找到的地方，按n可以定位到下一个匹配位置（按N定位到上一个）
:1,$s/hadoop/root/g 将第一行到追后一行的hadoop替换为root
:1,$s/hadoop/root/c 将第一行到追后一行的hadoop替换为root(有提示)

# 另存与追加
:w a.txt另存为
:w >> a.txt内容追加到a.txt

# 恢复到最初状态
:e!
```

## 文件夹属性
```shell
# 查看 aaa 文件夹属性
ls -ld aaa/
```
```
--x:可以cd进去
r-x:可以cd进去并ls
-wx:可以cd进去并touch，rm自己的文件，并且可以vi其他用户的文件
-wt:可以cd进去并touch，rm自己的文件

# 查看文件信息
file filename

# 查看文件详情
stat filename

# 查看文件大小
du -h   # 当前文件大小
du -ah  # 列出所有子节点
```

## 文件权限操作

```
linux 文件权限的描述格式
drwxr-xr-x      （也可以用二进制表示  111 101 101  -->  755）

d：标识节点类型（d：文件夹   -：文件  l:链接）
r：可读   w：可写    x：可执行 
第一组rwx：  表示这个文件的拥有者对它的权限：可读可写可执行
第二组r-x：  表示这个文件的所属组对它的权限：可读，不可写，可执行
第三组r-x：  表示这个文件的其他用户（相对于上面两类用户）对它的权限：可读，不可写，可执行
```

``` shell
# 修改文件权限
chmod g-rw aa.1    # 表示将 aa.1 对所属组的 rw 权限取消
chmod o-rw aa.1 	 # 表示将 aa.1 对其他人的 rw 权限取消
chmod u+x aa.1     # 表示将 aa.1 对所属用户的权限增加 x

# 用数字的方式来修改权限
chmod 664 aa.1     # 修改成   rw-rw-r--

# 加 -R 参数，统一修改一个文件夹的所有内容的权限
chmod -R 770 aaa/
# 将当前目录下的所有文件与子目录皆设为任何人可读取:
chmod -R a+r *  # * 表示当前路径

# 修改文件所属用户
chown aurelius:aurelius aaa/    # 只有root能执行

# 设为其拥有者和所在组可写入，但其他以外的人则不可写入
chmod ug+w,o-w a.txt b.txt

# 所有人都可以写和执行
chmod a=wx c.txt
chmod 333 c.txt

# 将/aaa目录下的所有文件与子目录皆设为任何人可读取
chmod -R a+r /aaa

# 将/aaa 目录下的所有文件与子目录的拥有者设为root，用户拥有组为users
chown -R root:users /aaa

# 将当前目录下的所有文件与子目录的用户皆设为 aurelius，组设为users
chown -R aurelius:users *
```

**目录没有执行权限的时候普通用户不能进入**

**文件只有读写权限的时候普通用户是可以删除的(删除文件不是修改它,是操作父及目录),只要父级目录有执行和修改的权限**

## 参考
[参考博客](https://www.cnblogs.com/stephen-liu74/archive/2011/11/10/2240461.html)

# 用户管理

## 添加用户
```shell
# 添加用户 aurelius
useradd aurelius  # 要修改密码才能登陆 
passwd aurelius  # 按提示输入密码

# 查看 aurelius 的组信息
groups aurelius

# 添加一个 america 组
groupadd america

# 添加一个tom用户，设置它属于users组，并添加注释信息
# 分步：
useradd tom
usermod -g users tom
usermod -c "hr tom" tom
# 一步：
useradd -g users -c "hr tom" tom
```

## 修改用户
``` shell
# 修改tom用户的登陆名为tomcat
usermod -l tomcat tom

# 将tomcat添加到 sys 和 root 组中
usermod -G sys,root tomcat

# 将jerry添加到america组中
usermod -g america jerry

# 将tomcat用户从root组和sys组删除
gpasswd -d tomcat root
gpasswd -d tomcat sys

# 将america组名修改为am
groupmod -n am america
```

## 配置sudo权限
```
# 用root编辑 vi /etc/sudoers
# 在文件的末尾，为 aurelius 添加一行：
aurelius  ALL=(ALL)       ALL
```
``` shell
# 然后，aurelius 用户就可以用 sudo 来执行系统级别的指令
sudo useradd abc
```

# 系统操作命令

## 查找
```shell
# 查找可执行的命令：
which ls

# 查找可执行的命令和帮助的位置：
whereis ls

# 查找文件(需要更新库:updatedb)
locate hadoop.txt

# 从某个文件夹开始查找
find / -name "hadooop*"
find / -name "hadooop*" -ls

# 查找并删除
find / -name "hadooop*" -ok rm {} \;
find / -name "hadooop*" -exec rm {} \;

# 查找用户为hadoop的文件
find /usr -user hadoop -ls

# 查找用户为hadoop并且(-a)拥有组为root的文件
find /usr -user hadoop -a -group root -ls

# 查找用户为hadoop或者(-o)拥有组为root并且是文件夹类型的文件
find /usr -user hadoop -o -group root -a -type d

# 查找权限为777的文件
find / -perm -777 -type d -ls

# 将当前目录及其子目录下所有扩展名是java的文件列出来。
find -name "*.java"  # 可以按文件名称搜索
# 查找当前目录及其子目录下扩展名是java 的文件
find -name *.java   # 只能按扩展名搜索

# 删除当前目录下扩展名是java的文件
rm -f *.java   # 不会删除子目录下的 .java 文件

# 显示命令历史
history

# grep
grep hadoop /etc/password

# 查看/etc下的文件有多少个？
ls -l /etc/ | grep '^d' | wc -l

# 查看/etc下的文件有多少个，并将文件详情输入到result.txt中
ls -l /etc/ | grep '^d' | tee result.txt | wc -l
```

## 进程控制
``` shell
# 查看进程
top

# 结束进程
kill 1234
kill -9 4333
```

## 后台服务管理
```shell
service network status   查看指定服务的状态
service network stop     停止指定服务
service network start    启动指定服务
service network restart  重启指定服务
service --status-all  查看系统中所有的后台服务

## 设置后台服务的自启配置
chkconfig   查看所有服务器自启配置
chkconfig iptables off   关掉指定服务的自动启动
chkconfig iptables on   开启指定服务的自动启动
```


## 系统启动级别管理
```shell
vi  /etc/inittab

# Default runlevel. The runlevels used are:
#   0 - halt (Do NOT set initdefault to this)
#   1 - Single user mode
#   2 - Multiuser, without NFS (The same as 3, if you do not have networking)
#   3 - Full multiuser mode
#   4 - unused
#   5 - X11
#   6 - reboot (Do NOT set initdefault to this)
#
id:3:initdefault:
```

## 时间与日历
``` shell
# 查看当前系统时间
date               # Thu Jul 25 11:53:12 EDT 2019
date +%Y-%m-%d        # 2019-07-25
date +%T            # 11:42:44
date +%Y-%m-%d" "%T     # 2019-07-25 11:45:16

# 日历
cal 2019
```

## 系统信息
```shell
# 查看有谁在线
who # 查看当前在线
last # 查看最近登录
lastlog

# 查看系统信息
uname -a # --all
uname -r # --kernel-release

# 查看ID命令
id -u aurelius # 用户id， 不指定则表示当前用户
id -g       # 组id

# 查看内存使用情况
free

# 查看文件系统
/proc

# 查看日志
ls /var/log/

# 查看系统报错日志
tail /var/log/messages
```

## 硬盘与分区
```shell
# 挂载硬盘
mount
umount
mount -t cifs //192.168.100.1/tools /mnt  # 加载windows共享

# 查看分区
df -h
```

## 名令别名(重启后无效)
``` shell
alias ll="ls -l"
# 取消
unalias ll
vi ~/.bashrc  # 修改后别名重启有效
```

## 虚拟机传文件
```shell
# sftp(alt+p调出)
# 上传
cd /home/
put C:\users\administrator\destop\1.txt
# 下载
lcd d:/ (指定下载到到目标路径)
get /home/1.txt
```

## iptables
```shell
# 查看帮助
iptables -h
man iptables

# 列出iptables规则
iptables -L -n
# 列出iptables规则并显示规则编号
iptables -L -n --line-numbers

列出iptables nat表规则（默认是filter表）
iptables -L -n -t nat

# 清除默认规则（注意默认是filter表，如果对nat表操作要加-t nat）
# 清楚所有规则
iptables -F 

# 重启iptables发现规则依然存在，因为没有保存
service iptables restart

# 保存配置
service iptables save

# 禁止ssh登陆（若果服务器在机房，一定要小心）
iptables -A INPUT -p tcp --dport 22 -j DROP
# 删除规则
iptables -D INPUT -p tcp --dport 22 -j DROP
# 注意：INPUT和DROP这样的关键字需要大写
```
```
-A, --append chain	追加到规则的最后一条
-D, --delete chain [rulenum]	Delete rule rulenum (1 = first) from chain
-I, --insert chain [rulenum]	Insert in chain as rulenum (default 1=first) 添加到规则的第一条
-p, --proto  proto	protocol: by number or name, eg. 'tcp',常用协议有tcp、udp、icmp、all
-j, --jump target 常见的行为有ACCEPT、DROP和REJECT三种，但一般不用REJECT，会带来安全隐患
```

```shell
# 禁止192.168.33.0网段从eth0网卡接入
iptables -A INPUT -p tcp -i eth0 -s 192.168.33.0 -j DROP
iptables -A INPUT -p tcp --dport 22 -i eth0 -s 192.168.33.61  -j ACCEPT

# 禁止ip地址非192.168.10.10的所有类型数据接入
iptables -A INPUT ! -s 192.168.10.10 -j DROP

# 禁止ip地址非192.168.10.10的ping请求
iptables -I INPUT -p icmp --icmp-type 8 -s 192.168.50.100 -j DROP

#匹配端口范围
iptables -I INPUT -p tcp --dport 22:80 -j DROP

#匹配多个端口
iptables -I INPUT -p tcp -m multiport --dport 22,80,3306 -j ACCEPT

#不允许源端口为80的数据流出
iptables -I OUTPUT -p tcp --sport 80 -j DROP
```

## ssh 远程登陆
``` shell
# 直接登陆
ssh aurelius@192.168.100.12

# 主机之间的免密ssh登陆
# A 登陆 B， 在A上操作：
# 首先生成密钥对
ssh-keygen   # (提示时，直接回车即可)
# 再将A自己的公钥拷贝并追加到B的授权列表文件authorized_keys中
ssh-copy-id B
```

## 关机
``` shell
shutdown -h now /init 0
shutdown -r now /reboot
```


## 快捷键
```
ctrl + c：停止进程
ctrl + l：清屏
ctrl + r：搜索历史命令
ctrl + q：退出
```

# 安装软件

## 安装 JDK
```shell
# 添加执行权限 
chmod u+x jdk-6u45-linux-i586.bin
# 解压
./jdk-6u45-linux-i586.bin
# 在/usr目录下创建java目录
mkdir /usr/java
# 将/soft目录下的解压的jdk1.6.0_45剪切到/usr/java目录下
mv jdk1.6.0_45/ /usr/java/
# 添加环境变量
vi /etc/profile
```

```
# 在/etc/profile文件最后添加
export JAVA_HOME=/usr/java/jdk1.6.0_45
export CLASSPATH=$JAVA_HOME/lib
export PATH=$PATH:$JAVA_HOME/bin
```

```shell
更新配置
source /etc/profile
```

## 安装tomcat
```shell
tar -zxvf /soft/apache-tomcat-7.0.47.tar.gz -C /programs/
cd /programs/apache-tomcat-7.0.47/bin/
./startup.sh
```

## 安装eclipse

# 本地 yum 仓库

## 本地
```
1. 挂载光盘到本地文件目录
mount -t iso9660 /dev/cdrom   /mnt/cdrom
为了避免每次重启后都要手动mount，可以在/etc/fstab中加入一行挂载配置，即可自动挂载
vi  /etc/fstab
/dev/cdrom              /mnt/cdrom              iso9660 defaults        0 0	
2. 配一个本地文件系统上的yum仓库 file:///packege/path/
```

## 局域网库
```
在局域网内部配置一台节点(server-base)的本地文件系统yum仓库，然后将其发布到web服务器中，其他节点就可以通过http://server-base/pagekege/path/
1. 挑选一台机器 node3，挂载一个系统光盘到本地目录/mnt/cdrom
2. 然后启动一个httpd服务器，将/mnt/cdrom 软连接到httpd服务器的/var/www/html目录中 (cd /var/www/html; ln -s /mnt/cdrom ./centos )
3. 通过网页访问测试一下：  http:// node3/centos   会看到光盘的目录内容
4. 在各台yum的客户端配置这个http地址到repo配置文件中
```